In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-70b-versatile",groq_api_key=groq_api_key)
model.invoke("Hello, how are you today?")

AIMessage(content="Hello, I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 42, 'total_tokens': 104, 'completion_time': 0.225454545, 'prompt_time': 0.008409018, 'queue_time': 0.07758452099999999, 'total_time': 0.233863563}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-48215e68-e8c9-43b3-8462-c591c555afe6-0', usage_metadata={'input_tokens': 42, 'output_tokens': 62, 'total_tokens': 104})

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, My name is Rahul and I am a Data Scientist")])

AIMessage(content='Hello Rahul, nice to meet you. As a Data Scientist, you must be working on some fascinating projects, analyzing complex data, and uncovering insights to drive business decisions. What specific areas of data science interest you the most, such as machine learning, natural language processing, or computer vision?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 47, 'total_tokens': 107, 'completion_time': 0.302656961, 'prompt_time': 0.015180511, 'queue_time': 1.078883723, 'total_time': 0.317837472}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-acaadfa9-a69b-4754-ae8d-88620c3ba2fc-0', usage_metadata={'input_tokens': 47, 'output_tokens': 60, 'total_tokens': 107})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hello, My name is Rahul and I am a Data Scientist"),
        AIMessage(content="Hello Rahul, nice to meet you. As a Data Scientist, you must be working on some fascinating projects, analyzing complex data and uncovering valuable insights. What kind of projects are you currently working on, or what areas of data science interest you the most?"),
        HumanMessage(content="What is my name?")
    ]
)

AIMessage(content='Your name is Rahul.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 114, 'total_tokens': 120, 'completion_time': 0.021818182, 'prompt_time': 0.019230544, 'queue_time': 0.000175034000000001, 'total_time': 0.041048726}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ae65402-7d12-4966-aa03-9129dd3c0070-0', usage_metadata={'input_tokens': 114, 'output_tokens': 6, 'total_tokens': 120})

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [7]:
config={
    "configurable":{"session_id":"chat1"}
}

In [8]:
respone=with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Rahul and I am a Data Scientist")],
    config=config
)

In [9]:
## changing the config --> session_id
config1={"configurable":{"session_id":"chat1"}}
respone=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)
respone.content

"Your name is Rahul, and you're a Data Scientist."

In [10]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability.",),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [11]:
chain.invoke({"messages":[HumanMessage(content="Hello, My name is Rahul and I am a Data Scientist")]})

AIMessage(content="Hello Rahul, nice to meet you. As a Data Scientist, you must be working on some fascinating projects, analyzing complex data, and uncovering valuable insights. What specific areas of data science interest you the most, such as machine learning, natural language processing, or visualization? Or what kind of projects are you currently working on? I'm here to help and chat about anything related to data science.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 64, 'total_tokens': 145, 'completion_time': 0.294545455, 'prompt_time': 0.012497764, 'queue_time': 0.000287486, 'total_time': 0.307043219}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-2a758018-a589-4639-bc72-20653cb5c06b-0', usage_metadata={'input_tokens': 64, 'output_tokens': 81, 'total_tokens': 145})

In [12]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [13]:
config={"configurable":{"session_id":"chat-1"}}

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Rahul and I am a Data Scientist")],
    config=config
)
response.content

"Hello Rahul, nice to meet you. As a Data Scientist, you must be working on some fascinating projects, analyzing complex data, and extracting valuable insights. What kind of projects are you currently working on, or what areas of data science interest you the most (e.g., machine learning, natural language processing, computer vision)? I'm here to help and chat about anything related to data science or beyond."

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt1=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability in {language}",),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain1=prompt1|model

In [16]:
respone1=chain1.invoke({"messages":[HumanMessage(content="Hello, My name is Rahul and I am a Data Scientist")],"language":"Hindi"})
respone1.content

'नमस्ते राहुल जी, मैं आपका सहायक हूँ। आपका डेटा साइंटिस्ट के रूप में काम करना बहुत रोचक होगा। डेटा साइंस में आपकी विशेषज्ञता क्या है, जैसे कि मशीन लर्निंग, डीप लर्निंग, या डेटा एनालिटिक्स?'

In [17]:
with_message_history=RunnableWithMessageHistory(
    chain1,
    get_session_history,
    input_messages_key="messages"
)

In [19]:
config1={"configurable":{"session_id":"chat-4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hello, My name is Rahul and I am a Data Scientist")],"language":"Hindi"},
    config=config1
)

In [20]:
respone.content

"Your name is Rahul, and you're a Data Scientist."